# Objective

### The goal of this data analysis project using sql would be to identify opportunities to increase occupancy rate on low-performing flights, which can ultimately lead to increased profitability for the airline.

# Importing libraries

In [2]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# DataBase Connection

In [3]:
connection = sqlite3.connect('travel.sqlite')
cursor = connection.cursor()

In [5]:
# This command return all the table name that is present in the database
cursor.execute("""select name from sqlite_master where type = 'table';""")
print("list of tables present in the database")
table_list = [table[0] for table in cursor.fetchall()]
table_list

list of tables present in the database


[]

# Data Exploration

In [7]:
# This pandas plus sql query return dataframe of all records that is available in the column so we can perform each and every method that is applicable on dataframe.
# Suppose there is one table name as aircraft_data so using below query we can see the all records in the form of dataframe.
aircraft_data = pd.read_sql_query("select * from aircraft_data", connection)
aircraft_data.head()


DatabaseError: Execution failed on sql 'select * from aircraft_data': no such table: aircraft_data

In [ ]:
#Using below query we can see the datatype of each column in each table that is present in the database.
for table in table_list:
    print('\ntable:', table)
    column_info = connection.execute("PRAGMA table_info({})".format(table))
    for column in column_info.fetchall()
    print(column[1:3])
    

In [ ]:
#using below query we can check the missing value in each column of each table that is present in the database
for table in table_list:
    print("\ntable:", table)
    df_table = pd.read_sql_query(f"select * from {table}", connection)
    print(df_table.isnull().sum())

# Basic Analysis

#### How many planes have more than 100 seats?

In [ ]:
# using below query we can check first question.
pd.read_sql_query("""select aricraft_code, count(*) as num_seats from seats
                        group by aircraft_code having num_seats > 100""")

#### How the number of tickets booked and total amount earned changed with time

# Using below query we can join two table into one 
tickets = pd.read_sql_query("""select * from tickets inner join bookings
                                on tickets.book_ref = bookings.book_ref""", connection)

# using below query we can change the datatype of book_date column
tickets['book_date'] = pd.to_datetime(tickets['book_date'])

# using below query we can fetch only date from book_date
tickets['date'] = pd.to_datetime(tickets['book_date'])

# using below query we can display how many tickets are booked per day
x = tickets.groupby('date')[['date']].count()

# now we are going to plot these thing in the form of line chart
plt.figure(figsize = (18,6))
plt.plot(x.index, x['date'], marker = '^')
plt.xlabel('Date', fontsize = 20)
plt.ylabel('Number of tickets', fontsize = 20)
plt.grid('b')
plt.show()


In [ ]:
# using below query we can display the sum of total amount per day
bookings = pd.read_sql_query("select * from bookings", connection)
bookings['book_date'] = pd.to_datetime(bookings['book_date'])
bookings['date'] = bookings['book_date'].dt.date
x = bookings.groupby('date')[['total_amount']].sum()
plt.figure(figsize = (18,6))
plt.plot(x.index, x['total_amount'], marker = '^')
plt.xlabel('Date', fontsize = 20)
plt.ylabel("Total Amount earned ", fontsize = 20)
plt.grid('b')
plt.show()


#### Calculate the average charges for each aircraft with different fare conditions

In [ ]:
# using below query we can achive above question

df = pd.read_sql_query("""select fare_condition, aircraft_code, avg(amount)
                            from ticket_flights join flights on ticket_flights.flight_id = flights.flight_id
                            group by aircraft_code, fare_condition""", connection)
sns.barplot(data = df, x = 'aircraft_code', y = 'avg(amount)', hue = 'fare_condition')

# Analyzing occupancy rate

#### For each aircraft, calculate the total revenue per year and the average revenue per ticket. 

In [ ]:
# using below query we can achive the above target
pd.read_sql_query("""select aircraft_code, total_revenue, ticket_count, total_revenue/ticket_count as avg_revenue_per_ticket from
(select aircraft_code, count(*) as ticket_count, sum(amount) as total_revenue from ticket_flights join flights on ticket_flights
.flight_id = flights.flight_id group by aircraft_code)""", connection)


# Calculate the average occupancy per aircraft.

In [ ]:
occupancy_rate = pd.read_sql_query("""select a.aircraft_code, avg(a.seats_count) as booked_seats, b.num_seats, avg(a.seats_count)/b.num_seats as occupancy_rate
from
(select aircraft_code, flights.flight_id, count(*) as seats_count from boarding_passes
inner join flights
on boarding_passes.flight_id = flights.flight_id
group by aircraft_code, flights.flight_id) as a
inner join 
(select aircraft_code, count(*) as num_seats from seats group by aircraft_code) as b
on a.aircraft_code = b.aircraft_code group by a.aircraft_code""", connection)

# Calculate by how much the total annual turnover could increase by giving all aircraft a 10% higher occupancy rate.

In [ ]:
# using below query we can achieve above target
occupancy_rate['Inc occupancy rate'] = occupancy_rate['occupancy_rate']+occupancy_rate['occupancy_rate']*0.1
occupancy_rate


In [ ]:
total_revenue = pd.read_sql_query("""select aircraft_code, sum(amount) as total_revenue from ticket_flights 
join flights on ticket_flights.flight_id = flights.flight_id group by aircraft_code""", connection)
total_revenue

In [ ]:
occupancy_rate['Inc total Annual Turnover'] = (total_revenue['total_revenue']/occupancy_rate['occupancy_rate'])*occupancy_rate['Inc occupancy rate']
occupancy_rate

In [ ]:
# Using below query we can see whole number
pd.set_option("display.float_format", str)